In [1]:
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import requests
import os
import re
from time import sleep
from dateutil.relativedelta import relativedelta
from datetime import datetime
import ujson as json
from random import shuffle
from newspaper import Article

headers = {'User-Agent': UserAgent()['Chrome']}

In [4]:
urls = set(urls)

query = "기후"
cur_date = start_date = datetime(2017, 11, 5)
end_date = datetime.today()

while cur_date <= end_date:
    date = (cur_date.year, cur_date.month, cur_date.day)
    date_string = "%d-%02d-%02d" % date
    date_string_ = "%d%02d%02d" % date

    page = 0
    while True:
        # 기후가 포함된 온난화 검색 결과
        # &query=온난화+%2B기후
        url = f'https://search.naver.com/search.naver?where=news&query={query}&ds={date_string}&de={date_string}&nso=so%3Ar%2Cp%3Afrom{date_string_}to{date_string_}%2Ca%3Aall&start={page*10+1}'
        res = requests.get(url, headers=headers)
        bs = BeautifulSoup(res.text)
        
        linkList = bs.find('ul', {"class": 'type01'}).findAll('dl')
        hrefs = [dl.find('dt').find('a')['href'] for dl in linkList]
        urls.update(hrefs)
        for links in linkList:
            try:
                urls.update([a['href'] for a in links.find('ul', {'class': 'relation_lst'}).findAll('a')])
            except:
                continue

        # 끝 페이지 다다르면 종료
        try:
            current_page = int(
                bs.find(
                    'div', {
                        'class': 'paging'}).find('strong').text)
        except:
            break
        if page > current_page:
            break
        
        page += 1
        sleep(0.5)

    print(f"{cur_date.strftime('%Y-%m-%d')} - {len(urls)}")
    cur_date += relativedelta(days=1)
    
urls = list(urls)

AttributeError: 'NoneType' object has no attribute 'findAll'

In [5]:
urls = list(urls)
shuffle(urls)

with open('urls.json', 'w') as f:
    json.dump(urls, f)